In [1]:
import os
import shutil
import random
from tqdm import tqdm

def split_dataset(source_root, target_root, split_ratio=0.8, seed=42):
    """
    将原始数据集按比例划分为 train 和 val

    Args:
        source_root (str): 原始数据集路径 (里面包含 1, 2, 3, 4 四个文件夹)
        target_root (str): 新的数据集路径 (将生成 train 和 val)
        split_ratio (float): 训练集比例，默认 0.8
        seed (int): 随机种子，保证每次划分结果一致
    """

    # 1. 设置随机种子 (保证可复现性，写论文必备)
    random.seed(seed)

    # 2. 检查源文件夹
    if not os.path.exists(source_root):
        print(f"[错误] 找不到源文件夹: {source_root}")
        return

    # 3. 准备目标文件夹
    train_dir = os.path.join(target_root, 'train')
    val_dir = os.path.join(target_root, 'val')

    # 如果目标文件夹存在，建议先手动删掉，防止混淆
    if os.path.exists(target_root):
        print(f"[警告] 目标文件夹 '{target_root}' 已存在，可能会导致文件重复！")

    # 获取所有类别 (1, 2, 3, 4)
    # 过滤掉非文件夹的杂项
    classes = [d for d in os.listdir(source_root) if os.path.isdir(os.path.join(source_root, d))]
    classes.sort() # 排序，保证处理顺序

    print(f"检测到类别: {classes}")
    print(f"开始划分... 训练集比例: {split_ratio}")

    # 4. 遍历每个类别进行划分
    for cls in classes:
        cls_path = os.path.join(source_root, cls)

        # 获取该类别下所有图片
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.bmp', '.jpg', '.png'))]

        #随机打乱
        random.shuffle(images)

        # 计算切割点
        split_point = int(len(images) * split_ratio)

        train_images = images[:split_point]
        val_images = images[split_point:]

        # 5. 执行复制
        # 创建对应的子文件夹，例如 dataset_split/train/1
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

        print(f"正在处理类别 [{cls}]: 总数 {len(images)} -> 训练 {len(train_images)} | 验证 {len(val_images)}")

        # 复制到 train
        for img in tqdm(train_images, desc=f"Copying {cls} to Train", leave=False):
            src = os.path.join(cls_path, img)
            dst = os.path.join(train_dir, cls, img)
            shutil.copy2(src, dst) # copy2 会保留文件的创建时间等元数据

        # 复制到 val
        for img in tqdm(val_images, desc=f"Copying {cls} to Val  ", leave=False):
            src = os.path.join(cls_path, img)
            dst = os.path.join(val_dir, cls, img)
            shutil.copy2(src, dst)

    print("\n" + "="*30)
    print("划分完成！新的数据集结构如下：")
    print(f"{target_root}/")
    print(f"  ├── train/ (包含 {classes})")
    print(f"  └── val/   (包含 {classes})")
    print("="*30)

if __name__ == '__main__':
    # --- 修改这里 ---
    # 你的原始文件夹 (里面有 1, 2, 3, 4)
    original_dataset_path = 'datasets'

    # 你想输出到哪里 (脚本会自动创建这个文件夹)
    new_dataset_path = 'datasets_split'

    split_dataset(original_dataset_path, new_dataset_path, split_ratio=0.8)

检测到类别: ['1', '2', '3', '4']
开始划分... 训练集比例: 0.8
正在处理类别 [1]: 总数 149 -> 训练 119 | 验证 30


正在处理类别 [2]: 总数 155 -> 训练 124 | 验证 31


正在处理类别 [3]: 总数 150 -> 训练 120 | 验证 30


正在处理类别 [4]: 总数 149 -> 训练 119 | 验证 30



划分完成！新的数据集结构如下：
datasets_split/
  ├── train/ (包含 ['1', '2', '3', '4'])
  └── val/   (包含 ['1', '2', '3', '4'])
